# Part 2: Automatic differentiation

In [1]:
import numpy as np

### Question 1: implement the relu function and its VJP in the format above. Using the finite difference equation (slide 13), make sure that the VJP is correct numerically.

In [2]:
def relu(x):
  """
  Args:
    x: an array

  Returns:
    - value of the function ReLU(x) 
    - function vjp to easily compute vjp of ReLU
  """
  value = np.maximum(x,0) #by defintion of the ReLU function 
  
  def vjp(u):
    relu_derivative = (x > 0) * 1 #by definition of the derivative and ReLU function 
    vjp_wrt_x = np.multiply(u,relu_derivative) #using slides 27 of the course
    return vjp_wrt_x,  

  return value, vjp

Now, we will create some functions to numerically check if the function defined above (and others ones later) are correct.

In [4]:
def test_vjp(f, x, u, eps=1e-3):
  """
  Args:
    f: a function returning an array.
    x: an array of size n 
    eps: numerical value (very small)
    u: an array of size m 

  Returns:
    numerical_vjp
  """
  
  def e(i): # to define each direction in the space
    basis_vector = np.zeros(len(x))
    basis_vector[i] = 1
    return basis_vector
  
  Jacobian = np.zeros((len(f(x)),len(x)))
  for i in range (len(x)):
    Jacobian[:,i] = (f(x + eps * e(i)) - f(x)) / eps #finite difference
  
  return np.dot(Jacobian.T,u)


We then test our implemented function

In [8]:
# we define some values for x and u, they must same dimension because for ReLU space of inputs and outputs are the same
x = np.linspace(start = -2, stop = 2, num = 40)
u = np.linspace(start = -5, stop = 2, num = 40)

def relu_function(x):
    return np.maximum(x,0)

relu_numerical_vjp = test_vjp(relu_function, x, u, eps=1e-3)
implemented_vjp = relu(x)[1](u)[0]

print("The numeric method gives for the VJP:")
print(relu_numerical_vjp)


The numeric method gives for the VJP:
[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.         -1.41025641 -1.23076923 -1.05128205 -0.87179487
 -0.69230769 -0.51282051 -0.33333333 -0.15384615  0.02564103  0.20512821
  0.38461538  0.56410256  0.74358974  0.92307692  1.1025641   1.28205128
  1.46153846  1.64102564  1.82051282  2.        ]


In [9]:
print("The implemented method gives for the VJP:")
print(implemented_vjp)

The implemented method gives for the VJP:
[-0.         -0.         -0.         -0.         -0.         -0.
 -0.         -0.         -0.         -0.         -0.         -0.
 -0.         -0.         -0.         -0.         -0.         -0.
 -0.         -0.         -1.41025641 -1.23076923 -1.05128205 -0.87179487
 -0.69230769 -0.51282051 -0.33333333 -0.15384615  0.02564103  0.20512821
  0.38461538  0.56410256  0.74358974  0.92307692  1.1025641   1.28205128
  1.46153846  1.64102564  1.82051282  2.        ]


The two methods give the same results indeed.

### Question 2: reusing dot and relu, implement a 2-layer MLP with a relu activation


In [10]:
#we import dot from the google doc
def dot(W, x):
  """
  Args:
    W: an a matrix of shape (n1,n)
    x: an array of shape n 

  Returns:
    - value of the function dot(W,x) 
    - function vjp to easily compute vjp of the dot product
  """
  value = np.dot(W, x)

  def vjp(u):
    return np.outer(u, x), np.dot(W.T,u)

  return value, vjp

In [11]:
def mlp2(x, W1, W2):
    """
    This function defines a MLP-2 architecure with ReLU function activation

    Args:
        x: an array of shape n 
        W1: an a matrix of shape (n1,n)
        W2: an a matrix of shape (1,n1)

    Returns:
        - value of the function mlp2(x, W1, W2)
        - function vjp to easily compute vjp of mlp2 archietcture 
    """

    x1 = dot(W1,x)[0] #1st linear layer 
    x2 = relu(x1)[0] #ReLU activation 
    x3 = dot(W2,x2)[0] #2nd linear layer
    value = x3

    def vjp(u):
        
        vjp_jacobian_dot_W2 = dot(W2,x2)[1](u) 
        vjp_wrt_W2 = vjp_jacobian_dot_W2[0] # we keep the 1st vjp, with respect to W2

        vjp_jacobian_dot_W2_wrt_x2 = vjp_jacobian_dot_W2[1] # we keep the 2nd vjp, with respect to x2
        vjp_jacobian_relu_wrt_x1 = relu(x1)[1](vjp_jacobian_dot_W2_wrt_x2)[0]

        vjp_jacobian_dot_W1 =  dot(W1,x)[1](vjp_jacobian_relu_wrt_x1) 
        vjp_wrt_W1 = vjp_jacobian_dot_W1[0] # we keep the 1st vjp, with respect to W1
        vjp_wrt_x = vjp_jacobian_dot_W1[1] # we keep the 2nd vjp, with respect to x
        
        return vjp_wrt_x,vjp_wrt_W1, vjp_wrt_W2

    return value, vjp

### Question 3: implement the squared loss VJP

In [12]:
def squared_loss(y_pred, y):
    """
    This function defines squared-loss between 2 vectors

    Args:
        y_pred: a scalar
        y: a scalar 
        
    Returns:
        - value of the function squared_loss(y_pred, y)
        - function vjp to easily compute vjp of the squared loss
    """

    residual = y_pred - y
    value = 0.5 * np.sum(residual ** 2)

    def vjp(u):
        vjp_y_pred = np.multiply(residual,u)
        vjp_y = -np.multiply(residual,u)
        return vjp_y_pred, vjp_y
    
    # The code requires every output to be an array.
    return np.array([value]), vjp

### Question 4: implement the loss by composing mlp2 and squared_loss


In [ ]:
def loss(x, y, W1, W2):
    """
    This function defines the loss, by combining previous function squared_loss and mlp2

    Args:
        x: an array of shape n 
        y: an array of shape n 
        W1: an a matrix of shape (n1,n)
        W2: an a matrix of shape (1,n1)
        
    Returns:
        - value of the function loss(x, y, W1, W2)
        - function vjp to easily compute vjp of the loss
    """

    y_pred = mlp2(x, W1, W2)[0]
    value = squared_loss(y_pred, y)[0]

    def vjp(u):
        
        vjp_squared_loss = squared_loss(y_pred, y)[1](u)
        vjp_wrt_y = vjp_squared_loss[1] # we keep the vjp with respect to y 

        vjp_mlp2 = mlp2(x, W1, W2)[1](vjp_squared_loss[0])

        vjp_wrt_W2 = vjp_mlp2[2] # we keep the vjp with respect to W2
        vjp_wrt_W1 = vjp_mlp2[1] # we keep the vjp with respect to W1
        vjp_wrt_x = vjp_mlp2[0] # we keep the vjp with respect to Wx
        
        return vjp_wrt_x, vjp_wrt_y, vjp_wrt_W1, vjp_wrt_W2

    return value, vjp

Now let's numerically test our architecture

In [17]:
#check the gradient for x coordinate 

def test_architecture_vjp(f,x,y,W1,W2,u, eps=1e-3): 

    """
    Args:
        f: a function returning an array.
        x: an array of size n 
        eps: numerical value (very small)
        u: an array of size m 

    Returns:
        numerical_vjp

    """
    
    def e(i): # to define each direction in the space
        basis_vector = np.zeros(len(x))
        basis_vector[i] = 1
        return basis_vector
  
    Jacobian = np.zeros((len(f(x,y,W1,W2)),len(x)))
    for i in range (len(x)):
        Jacobian[:,i] = (f(x + eps * e(i),y,W1,W2) - f(x,y,W1,W2)) / eps #finite difference
  
    return np.dot(Jacobian.T,u) 
    

In [26]:
# we define some values for x and u, they must same dimension because for ReLU space of inputs and outputs are the same
m = 5
k = 10
x = np.linspace(start = -2, stop = 2, num = 40)
u = np.random.rand(1) #size 1 beacuse f is in value of scalar
W1 = np.random.rand(m,len(x))
W2 = np.random.rand(k,m)
y = np.linspace(start = -2, stop = 2, num = k)


def mlp2_function(x,y,W1,W2):
    x1 = dot(W1,x)[0] #1st linear layer 
    x2 = relu(x1)[0] #ReLU activation 
    y_pred = dot(W2,x2)[0] #2nd linear layer
    residual = y_pred - y
    value = 0.5 * np.sum(residual ** 2)
    return np.array([value])

relu_numerical_vjp = test_architecture_vjp(mlp2_function,x,y,W1,W2,u, eps=1e-3)
implemented_vjp = relu(x)[1](u)[0]

print("The numeric method gives for the VJP:")
print(relu_numerical_vjp)

The numeric method gives for the VJP:
[ 3.45052383  2.66317577  8.78690297  5.96159959  4.42238155  9.11140057
  6.82954782  3.20253968 12.33649434  3.85880904  4.85426372  6.61494078
  2.53303104  5.06409083  7.5592683  10.02295287  8.24660279  7.46863938
  8.20774187  9.41294542  5.28011901  8.09393933  2.29111948  7.73468241
  9.47269964  2.09238798  6.47378223  5.19637709  8.21736629  5.89765486
  6.15299244  5.86131837  9.17960146  9.3161047   7.34579066  9.54217488
  7.17141859  5.87332171  9.05419728  3.43960389]
